# Embedding 创建示例
<img src="/Users/caojm/course/学习笔记/AI/大模型/images/Embedding模型图解.svg" alt="Embedding模型图解" style="zoom:150%;" />

In [ ]:
from dotenv import load_dotenv
import os
from openai import OpenAI
import tiktoken 
import pandas as pd
import time

#初始化OpenAI客户端
load_dotenv()   # 从环境变量中获取API密钥
api_key = os.getenv('XIAOAI_API_KEY')
base_url = os.getenv('XIAOAI_BASE_URL')
client = OpenAI(api_key=api_key, base_url=base_url)


#用pandas 读取datas 下的文件
df = pd.read_csv('./datas/fine_food_reviews_1k.csv', index_col=0)
df = df[['Time', 'ProductId', 'UserId', 'Score', 'Summary', 'Text']]
df = df.dropna() # 删除cvs的NAN值
summary = df['Summary'].str.strip()  
text = df['Text'].str.strip()
df['combined'] = 'Title: ' + df['Summary'].str.strip() + 'Content: ' + df['Text'].str.strip()
# 打印前5条数据
# print(df.head(5))


# 创建分词器，并计算每一行token的数量
tokenizer = tiktoken.encoding_for_model("text-embedding-ada-002")  # 移除多余的分号
token_num = df['combined'].apply(lambda x: len(tokenizer.encode(x)))
df['n_tokens'] = token_num

# 截断文本使其不超过最大token数
token_max = 8191
# print(df[df['n_tokens'] > token_max].head(5)) # 打印 超过token限制的前5行数据

def batch_embedding_text(texts, batch_size=20, model='text-embedding-ada-002'):
    """
    通过OpenAI的Embedding模型处理文本数据，包含重试机制
    :param text: 需要处理的文本数据
    :param model: 使用的embedding模型
    :param retry_count: 重试次数
    :return: embedding向量
    """
    all_embeddings = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i + batch_size]
        try:
            resp = client.embeddings.create(input=batch, model=model)
            batch_embeddings = [data.embedding for data in resp.data]
            all_embeddings.extend(batch_embeddings)
            time.sleep(0.5)  # 添加短暂延迟，避免触发速率限制
        except Exception as e:
            print(f"批处理出错 ({i} to {i+batch_size}): {str(e)}")
            all_embeddings.extend([None] * len(batch))
            time.sleep(1)  # 出错时等待更长时间
    return all_embeddings

# 创建embeddings 只处理最后1000条数据
df_sample = df[df['n_tokens'] <= token_max].tail(1000)  
df_sample['embedding'] = batch_embedding_text(df_sample['combined'].tolist())
df_sample = df_sample.dropna(subset=['embedding'])  # 移除embedding为None的行（如果有的话）

# 保存结果
df_sample.to_csv('./datas/embedding_output_caojm.csv')
print(f"成功处理的数据条数: {len(df_sample)}")


## 代码说明

```python
df['combined'] = f'''Title: {summary}Content: {text}'''
# 这里的 summary 和 text 都是 Series，不能直接用在 f-string 中。应该改为：
df['combined'] = df.apply(lambda row: f'''Title: {row['Summary']}Content: {row['Text']}''', axis=1)
```

- 那既然是Series对象，为何字符串拼接久可以呢？
在 Pandas 中，Series 对象支持向量化操作，包括字符串拼接。当你对 Series 进行字符串操作时，操作会自动应用到 Series 中的每个元素上。这是 Pandas 的一个重要特性。

```python
# 1. 使用 + 进行字符串拼接（向量化操作）
df['combined'] = 'Title: ' + df['Summary'].str.strip() + 'Content: ' + df['Text'].str.strip()

# 2. 使用 f-string（这样是错误的，因为f-string不支持向量化操作）
df['combined'] = f'''Title: {summary}Content: {text}'''  # 这样不行！

# 3. 正确的f-string用法（使用apply进行逐行操作）
df['combined'] = df.apply(lambda row: f"Title: {row['Summary']}Content: {row['Text']}", axis=1)
```

## 几个嵌入模型

1. text-embedding-3-large

- 功能： 这是较大的嵌入模型，相较于 smaller 版本，它通常在处理能力、语义理解精度等方面表现更好。
- 特点：
    - 更高精度： large 版本通常能够提供更高的语义理解能力，因此它在处理更复杂文本时的效果更好。
    - 计算需求更高： 与 smaller 版本相比，large 需要更多的计算资源（包括更长的响应时间和更高的成本）。
    - 适用场景： 适合需要高精度文本嵌入任务，如复杂文本相似度计算、语义搜索等。

2. text-embedding-3-small

- 功能： 这是较小的嵌入模型，相比于 large 版本，它更注重速度和计算效率，适用于对精度要求不太高但需要高效处理大规模数据的场景。
- 特点：
    - 更低精度： 相对于 large 版本，small 的精度较低，可能无法捕捉文本中所有的细节和复杂的语义关系。
    - 计算需求较低： 处理速度更快，对资源消耗更少，适合快速生成大量嵌入向量。
    - 适用场景： 适用于快速原型开发、实时搜索引擎等需要处理大量文本数据，但对精度要求相对较低的任务。
3. text-embedding-ada-002
- 功能： 这是OpenAI最近推出的嵌入模型，具有较低的成本和较好的性能。适合一般的文本嵌入任务，广泛用于文本相似性、搜索、信息检索和语义匹配等。
- 特点：
    - 高效：相比以前的模型，它的成本低，速度更快，且对文本处理能力较强。
    - 多用途：适用于各种嵌入任务，包括情感分析、语义搜索等。

## python 几个常用的数据科学库 

### 1. **tiktoken**

- **用途**：`tiktoken` 是一个用于处理 Token 化（tokenization）任务的库，特别是在处理与大语言模型（如 GPT）相关的任务时。
- **常见应用**：它用于将文本转换成模型可以理解的 tokens（词元），并且支持高效的文本编码和解码。通常在处理自然语言处理（NLP）任务时，尤其是对 GPT 系列模型进行文本预处理时会使用这个库。

### 2. **matplotlib**

- **用途**：`matplotlib` 是一个非常流行的 2D 图形绘制库，常用于数据可视化。
- **常见应用**：它可以帮助你创建静态、动态、交互式的图表和图像，如折线图、散点图、条形图等。`matplotlib` 允许你对图表进行高度定制，广泛用于科学计算和数据分析领域。
- **示例**：绘制数据趋势图、统计图表等。

### 3. **plotly**

- **用途**：`plotly` 是一个用于创建交互式图表的库，可以生成 2D 和 3D 图表。
- **常见应用**：与 `matplotlib` 不同，`plotly` 的图表具有高度交互性，可以在网页中进行缩放、拖动等操作，非常适合数据展示和分析。它支持创建更复杂和美观的图形，例如交互式的柱状图、饼图、热图、地图等。
- **示例**：制作交互式的可视化仪表板，展示数据分析结果。

### 4. **scikit-learn**

- **用途**：`scikit-learn` 是一个非常流行的机器学习库，提供了广泛的算法和工具，适用于分类、回归、聚类、降维等任务。
- **常见应用**：它包括许多常用的机器学习模型，如支持向量机（SVM）、决策树、随机森林、KNN、逻辑回归等，同时也提供了数据预处理、特征选择、模型评估等功能。
- **示例**：使用 `scikit-learn` 训练机器学习模型并进行预测、数据分割和评估。

### 5. **numpy**

- **用途**：`numpy` 是 Python 中用于处理大规模数值计算的基础库，特别是对数组和矩阵进行高效操作。
- **常见应用**：它提供了一个强大的 `ndarray` 对象，用于存储和操作多维数组。`numpy` 支持大规模的数学运算，如线性代数、傅里叶变换、随机数生成等，广泛应用于科学计算和数据分析中。
- **示例**：执行矩阵运算、计算统计量、处理大量数据集。

### 6. Pandas 

是 Python 中用于数据处理和分析的非常强大的库，特别适用于结构化数据（如表格数据）。它的主要功能是提供快速、灵活、表达力强的数据结构，特别是 DataFrame 和 Series，来简化数据清洗、探索和分析的工作。

### 主要功能和应用：

1. **DataFrame 和 Series 数据结构**：
   - **DataFrame**：一个二维表格数据结构，类似于数据库中的表或 Excel 中的工作表。它允许按行和列索引数据，并且每列可以有不同的类型（整数、浮动、字符串等）。
   - **Series**：一个一维数据结构，类似于数组或列表，通常用于表示单列数据。
2. **数据清洗和预处理**：
   - **处理缺失数据**：Pandas 提供了便捷的方法来填补或删除缺失值（如 `NaN`）。
   - **数据过滤和选择**：通过索引和条件筛选数据，可以非常容易地选择和过滤出需要的数据。
   - **数据变换**：可以对数据进行归一化、标准化、映射等变换操作。
3. **数据操作**：
   - **合并与连接**：Pandas 支持多种方式合并不同的数据集（例如 `concat`、`merge`），就像数据库中的连接操作。
   - **分组和聚合**：使用 `groupby` 可以将数据按某个或多个列进行分组，然后进行汇总统计（例如计算平均值、最大值、计数等）。
   - **排序**：可以根据一个或多个列对数据进行排序。
4. **数据读取与存储**：
   - **支持多种数据格式**：Pandas 可以读取和写入多种文件格式，如 CSV、Excel、JSON、SQL 等。
   - **快速读取大数据集**：Pandas 提供了高效的读取大数据集的功能，适合处理海量数据。
5. **数据可视化**：
   - Pandas 自带了一些基本的数据可视化功能，虽然它不如 `matplotlib` 或 `plotly` 强大，但可以快速地绘制数据图表（如折线图、条形图等），对数据进行初步可视化。

### 示例代码：

```python
import pandas as pd

# 创建一个 DataFrame
data = {
    'Name': ['Alice', 'Bob', 'Charlie', 'David'],
    'Age': [25, 30, 35, 40],
    'City': ['New York', 'Los Angeles', 'Chicago', 'Houston']
}

df = pd.DataFrame(data)

# 查看前几行数据
print(df.head())

# 选择一列
print(df['Name'])

# 筛选数据：筛选年龄大于 30 的人
filtered_df = df[df['Age'] > 30]
print(filtered_df)

# 数据分组并计算平均年龄
grouped = df.groupby('City')['Age'].mean()
print(grouped)
```

### 总结：

- **`tiktoken`**：处理文本 Token 化，通常与 NLP 和大模型相关。

- **`matplotlib`**：用于创建静态图表，常用于数据可视化。

- **`plotly`**：用于创建交互式图表，适合展示动态数据和分析结果。

- **`scikit-learn`**：机器学习库，用于构建、训练和评估机器学习模型。

- **`numpy`**：用于高效数值计算和数组操作，是数据分析和科学计算的基础库。

- **`Pandas `** 是一个非常适合数据分析和处理的库，特别是在处理和清理结构化数据时，它提供了高效、简便的操作。它在数据科学、机器学习、金融分析等领域非常常用，几乎是任何数据分析项目的基础工具之一。如果你处理的主要是表格数据或时间序列数据，Pandas 会极大地提高你的效率。

  <mark>如果你正在进行数据清洗、处理和分析，Pandas 是一个非常重要的工具！</mark>

这些库在数据科学、机器学习、自然语言处理等领域中都是非常重要的工具，您可以根据项目的需求选择适合的库